In [3]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D, concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import losses
from keras.utils import to_categorical
import keras.backend as K

import matplotlib.pyplot as plt

import numpy as np

Using TensorFlow backend.


In [4]:
from tensorflow.python.keras.backend import set_session
from tensorflow.python.keras.models import load_model

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

sess = tf.Session()
graph = tf.get_default_graph()

# IMPORTANT: models have to be loaded AFTER SETTING THE SESSION for keras! 
# Otherwise, their weights will be unavailable in the threads after the session there has been set
set_session(sess)

Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
class BIGAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # Build the encoder
        self.encoder = self.build_encoder()

        # The part of the bigan that trains the discriminator and encoder
        self.discriminator.trainable = False

        # Generate image from sampled noise
        z = Input(shape=(self.latent_dim, ))
        img_ = self.generator(z)

        # Encode image
        img = Input(shape=self.img_shape)
        z_ = self.encoder(img)

        # Latent -> img is fake, and img -> latent is valid
        fake = self.discriminator([z, img_])
        valid = self.discriminator([z_, img])

        # Set up and compile the combined model
        # Trains generator to fool the discriminator
        self.bigan_generator = Model([z, img], [fake, valid])
        self.bigan_generator.compile(loss=['binary_crossentropy', 'binary_crossentropy'],
            optimizer=optimizer)


    def build_encoder(self):
        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(self.latent_dim))

        model.summary()

        img = Input(shape=self.img_shape)
        z = model(img)

        return Model(img, z)

    def build_generator(self):
        model = Sequential()

        model.add(Dense(512, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        z = Input(shape=(self.latent_dim,))
        gen_img = model(z)

        return Model(z, gen_img)

    def build_discriminator(self):

        z = Input(shape=(self.latent_dim, ))
        img = Input(shape=self.img_shape)
        d_in = concatenate([z, Flatten()(img)])

        model = Dense(1024)(d_in)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.5)(model)
        model = Dense(1024)(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.5)(model)
        model = Dense(1024)(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.5)(model)
        validity = Dense(1, activation="sigmoid")(model)

        return Model([z, img], validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):


            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Sample noise and generate img
            z = np.random.normal(size=(batch_size, self.latent_dim))
            imgs_ = self.generator.predict(z)

            # Select a random batch of images and encode
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
            z_ = self.encoder.predict(imgs)

            # Train the discriminator (img -> z is valid, z -> img is fake)
            d_loss_real = self.discriminator.train_on_batch([z_, imgs], valid)
            d_loss_fake = self.discriminator.train_on_batch([z, imgs_], fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (z -> img is valid and img -> z is is invalid)
            g_loss = self.bigan_generator.train_on_batch([z, imgs], [valid, fake])

            # Plot the progress
            print ("%d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0]))

            # If at save interval => save generated image samples
            if epoch>=39000:
                self.sample_interval(epoch)

    def sample_interval(self, epoch):
        r, c = 5, 5
        z = np.random.normal(size=(25, self.latent_dim))
        gen_imgs = self.generator.predict(z)

        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig(r"C:\Users\rishi\Documents\BiGAN Images\mnist_%d.png" % epoch)
        plt.close()



In [6]:
global sess
global graph
with graph.as_default():
    set_session(sess)
    bigan = BIGAN()
    bigan.train(epochs=40000, batch_size=32, sample_interval=400)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 512)               51712     
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dense_6 (Dense)              (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_______________________________________

c:\users\rishi\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.142565, acc: 20.31%] [G loss: 3.788053]
1 [D loss: 0.401838, acc: 76.56%] [G loss: 4.805927]


c:\users\rishi\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


2 [D loss: 0.305574, acc: 87.50%] [G loss: 5.491858]
3 [D loss: 0.172577, acc: 98.44%] [G loss: 5.432396]
4 [D loss: 0.142342, acc: 98.44%] [G loss: 6.813780]
5 [D loss: 0.107257, acc: 98.44%] [G loss: 7.531394]
6 [D loss: 0.067374, acc: 100.00%] [G loss: 7.933378]
7 [D loss: 0.048581, acc: 100.00%] [G loss: 8.235534]
8 [D loss: 0.029633, acc: 100.00%] [G loss: 9.507342]
9 [D loss: 0.025062, acc: 100.00%] [G loss: 10.076907]
10 [D loss: 0.016633, acc: 100.00%] [G loss: 10.544199]
11 [D loss: 0.020430, acc: 100.00%] [G loss: 10.350292]
12 [D loss: 0.021242, acc: 100.00%] [G loss: 11.369394]
13 [D loss: 0.019686, acc: 100.00%] [G loss: 11.572286]
14 [D loss: 0.013105, acc: 100.00%] [G loss: 12.433326]
15 [D loss: 0.010323, acc: 100.00%] [G loss: 11.635399]
16 [D loss: 0.015095, acc: 100.00%] [G loss: 12.770947]
17 [D loss: 0.007836, acc: 100.00%] [G loss: 13.629477]
18 [D loss: 0.009017, acc: 100.00%] [G loss: 13.134753]
19 [D loss: 0.010645, acc: 100.00%] [G loss: 13.711994]
20 [D loss:

...

39303 [D loss: 0.709120, acc: 56.25%] [G loss: 1.815153]
39304 [D loss: 0.688253, acc: 51.56%] [G loss: 1.782396]
39305 [D loss: 0.671364, acc: 57.81%] [G loss: 1.788104]
39306 [D loss: 0.545196, acc: 71.88%] [G loss: 2.005568]
39307 [D loss: 0.640280, acc: 65.62%] [G loss: 1.880794]
39308 [D loss: 0.597546, acc: 67.19%] [G loss: 2.110167]
39309 [D loss: 0.664091, acc: 53.12%] [G loss: 2.052830]
39310 [D loss: 0.693433, acc: 57.81%] [G loss: 1.690266]
39311 [D loss: 0.635240, acc: 64.06%] [G loss: 1.807081]
39312 [D loss: 0.686655, acc: 59.38%] [G loss: 2.020942]
39313 [D loss: 0.662258, acc: 50.00%] [G loss: 1.963483]
39314 [D loss: 0.711460, acc: 56.25%] [G loss: 1.923444]
39315 [D loss: 0.662398, acc: 57.81%] [G loss: 1.863260]
39316 [D loss: 0.608672, acc: 64.06%] [G loss: 2.064733]
39317 [D loss: 0.735546, acc: 48.44%] [G loss: 1.664243]
39318 [D loss: 0.639518, acc: 60.94%] [G loss: 1.992900]
39319 [D loss: 0.689631, acc: 70.31%] [G loss: 2.019561]
39320 [D loss: 0.620677, acc: 5

39447 [D loss: 0.679816, acc: 60.94%] [G loss: 1.884444]
39448 [D loss: 0.736435, acc: 50.00%] [G loss: 1.981393]
39449 [D loss: 0.651014, acc: 64.06%] [G loss: 2.005973]
39450 [D loss: 0.656022, acc: 60.94%] [G loss: 1.848179]
39451 [D loss: 0.683157, acc: 59.38%] [G loss: 1.775069]
39452 [D loss: 0.633391, acc: 60.94%] [G loss: 1.912475]
39453 [D loss: 0.646097, acc: 62.50%] [G loss: 1.892483]
39454 [D loss: 0.622721, acc: 65.62%] [G loss: 1.940747]
39455 [D loss: 0.654255, acc: 51.56%] [G loss: 1.994282]
39456 [D loss: 0.684985, acc: 57.81%] [G loss: 1.891539]
39457 [D loss: 0.626055, acc: 60.94%] [G loss: 1.836121]
39458 [D loss: 0.675628, acc: 59.38%] [G loss: 1.757936]
39459 [D loss: 0.662907, acc: 60.94%] [G loss: 1.741038]
39460 [D loss: 0.611834, acc: 64.06%] [G loss: 2.026937]
39461 [D loss: 0.622081, acc: 64.06%] [G loss: 1.865222]
39462 [D loss: 0.653745, acc: 60.94%] [G loss: 1.728595]
39463 [D loss: 0.620227, acc: 65.62%] [G loss: 1.673246]
39464 [D loss: 0.641430, acc: 6

39591 [D loss: 0.678054, acc: 56.25%] [G loss: 2.142318]
39592 [D loss: 0.638898, acc: 59.38%] [G loss: 1.817968]
39593 [D loss: 0.611087, acc: 64.06%] [G loss: 1.937647]
39594 [D loss: 0.692699, acc: 53.12%] [G loss: 1.732652]
39595 [D loss: 0.694062, acc: 54.69%] [G loss: 1.894003]
39596 [D loss: 0.673990, acc: 56.25%] [G loss: 1.961610]
39597 [D loss: 0.697716, acc: 56.25%] [G loss: 1.917570]
39598 [D loss: 0.636118, acc: 60.94%] [G loss: 1.915854]
39599 [D loss: 0.730812, acc: 56.25%] [G loss: 1.980083]
39600 [D loss: 0.595557, acc: 71.88%] [G loss: 1.994506]
39601 [D loss: 0.653482, acc: 64.06%] [G loss: 2.102494]
39602 [D loss: 0.656178, acc: 65.62%] [G loss: 1.805098]
39603 [D loss: 0.620948, acc: 67.19%] [G loss: 1.840614]
39604 [D loss: 0.607843, acc: 68.75%] [G loss: 1.946250]
39605 [D loss: 0.635530, acc: 60.94%] [G loss: 2.221350]
39606 [D loss: 0.600971, acc: 60.94%] [G loss: 2.253600]
39607 [D loss: 0.624457, acc: 67.19%] [G loss: 1.841312]
39608 [D loss: 0.681174, acc: 5

39735 [D loss: 0.629375, acc: 65.62%] [G loss: 1.834360]
39736 [D loss: 0.607728, acc: 59.38%] [G loss: 1.964534]
39737 [D loss: 0.681285, acc: 59.38%] [G loss: 1.752831]
39738 [D loss: 0.665149, acc: 65.62%] [G loss: 1.976373]
39739 [D loss: 0.670155, acc: 56.25%] [G loss: 1.755373]
39740 [D loss: 0.696381, acc: 53.12%] [G loss: 1.618001]
39741 [D loss: 0.734525, acc: 60.94%] [G loss: 1.783075]
39742 [D loss: 0.598563, acc: 65.62%] [G loss: 2.026837]
39743 [D loss: 0.636777, acc: 56.25%] [G loss: 1.836374]
39744 [D loss: 0.602217, acc: 64.06%] [G loss: 1.809511]
39745 [D loss: 0.723016, acc: 60.94%] [G loss: 1.981767]
39746 [D loss: 0.646834, acc: 60.94%] [G loss: 1.978679]
39747 [D loss: 0.565785, acc: 73.44%] [G loss: 2.006003]
39748 [D loss: 0.618691, acc: 59.38%] [G loss: 2.210492]
39749 [D loss: 0.698221, acc: 57.81%] [G loss: 2.081917]
39750 [D loss: 0.703674, acc: 53.12%] [G loss: 1.972905]
39751 [D loss: 0.607177, acc: 67.19%] [G loss: 1.895432]
39752 [D loss: 0.656488, acc: 6

39879 [D loss: 0.800938, acc: 45.31%] [G loss: 1.619834]
39880 [D loss: 0.655804, acc: 57.81%] [G loss: 1.843117]
39881 [D loss: 0.567900, acc: 65.62%] [G loss: 1.815441]
39882 [D loss: 0.676113, acc: 60.94%] [G loss: 1.909135]
39883 [D loss: 0.632807, acc: 64.06%] [G loss: 1.956709]
39884 [D loss: 0.674354, acc: 53.12%] [G loss: 1.846199]
39885 [D loss: 0.570055, acc: 76.56%] [G loss: 2.338541]
39886 [D loss: 0.593545, acc: 68.75%] [G loss: 2.369192]
39887 [D loss: 0.650897, acc: 62.50%] [G loss: 2.082214]
39888 [D loss: 0.654356, acc: 64.06%] [G loss: 2.054824]
39889 [D loss: 0.665643, acc: 56.25%] [G loss: 1.838715]
39890 [D loss: 0.649477, acc: 67.19%] [G loss: 2.080622]
39891 [D loss: 0.619758, acc: 60.94%] [G loss: 1.884230]
39892 [D loss: 0.534729, acc: 79.69%] [G loss: 2.083777]
39893 [D loss: 0.673157, acc: 60.94%] [G loss: 1.916688]
39894 [D loss: 0.719638, acc: 48.44%] [G loss: 1.929277]
39895 [D loss: 0.689133, acc: 59.38%] [G loss: 1.989918]
39896 [D loss: 0.641281, acc: 6

In [ ]:
def generate_images(cnt):
    c=1
    while (c<=cnt):
        z = np.random.normal(size=(25, bigan.latent_dim))
        gen_imgs = bigan.generator.predict(z)
        gen_imgs = 0.5 * gen_imgs + 0.5
        plt.imshow(gen_imgs[0, :, :, 0], cmap='gray')
        plt.axis('off')
        plt.savefig(r"C:\Users\rishi\Documents\BiGAN Images\generated_bigan\bigan_{}.jpg".format(c), bbox_inches='tight', pad_inches=-0.0005)
        print("Image {} saved.".format(c))
        c+=1
        plt.close()
generate_images(10000)